In [1]:
import numpy as np

> - Four lettersL ICAO code of station identifier
>- Issuance Time (7 letters)
>> - in the from DDHHMMZ
>>  - DD= , HH= MM=
>- Wind (can vary in length)
>>- first 3 digits: wind direction
>>- next 2 digits mean speed
>>- optional G for gus
>>>- followed by 2 or 3 digit maximum speed
>>-next 2-3 digits units
>>>-KT=knots, KMH=km/hr, MPS=m/s
>- if wind direction varies 60 degrees or more, followed by 3 digits V 3 digits
>- prevailing visibility in statue miles: number SM
>>- runway visiual range R number L/R/C / P/M 4 digit value FT/M V / U/D/D
>>>- P=plus, M=minus, V=variability, U=up, D=down, N=no change
>- present weather chart with lots of options
>- sky cover
>>- SKC=sky clear, NSC=no significant clouds, FEW=few, SCT=scattered, BKN=broken, OVC=overcast
>- temperature/dewpoint (2 digit temps/2 digit dp) (M for minus)
>- altimeter setting and indicator

note ZULU time (24 hour time in UTC), use knots, directions in degrees

In [2]:
import metpy #install metpy using conda -- I personally had erros when using pip

In [3]:
from metpy.units import units

In [1]:
from metpy.io import parse_metar_to_dataframe

# Example METAR data
metar_data = 'METAR KATL 112351Z 09004KT 10SM FEW250 13/M02 A3013 RMK AO2 SLP204 T01281017 10139 20128 51006'

# Parse METAR data to a pandas DataFrame
df = parse_metar_to_dataframe(metar_data)

# Display the DataFrame
df


,station_id,latitude,longitude,elevation,date_time,wind_direction,wind_speed,wind_gust,visibility,current_wx1,...,air_temperature,dew_point_temperature,altimeter,current_wx1_symbol,current_wx2_symbol,current_wx3_symbol,remarks,air_pressure_at_sea_level,eastward_wind,northward_wind
station_id,,,,,,,,,,,,,,,,,,,,,
KATL,KATL,33.63,-84.45,296,2023-11-11 23:51:00,90,4.0,NaN,16093.44,NaN,...,13.0,-2.0,30.13,0,0,0,AO2 SLP204 T01281017 10139 20128 51006,1020.79,-4.0,-2.449294e-16


In [7]:
df.columns

Index(['station_id', 'latitude', 'longitude', 'elevation', 'date_time',
       'wind_direction', 'wind_speed', 'wind_gust', 'visibility',
       'current_wx1', 'current_wx2', 'current_wx3', 'low_cloud_type',
       'low_cloud_level', 'medium_cloud_type', 'medium_cloud_level',
       'high_cloud_type', 'high_cloud_level', 'highest_cloud_type',
       'highest_cloud_level', 'cloud_coverage', 'air_temperature',
       'dew_point_temperature', 'altimeter', 'current_wx1_symbol',
       'current_wx2_symbol', 'current_wx3_symbol', 'remarks',
       'air_pressure_at_sea_level', 'eastward_wind', 'northward_wind'],
      dtype='object')

In [ ]:
import datetime
from urllib.request import urlopen
import json
from IPython.display import clear_output

# Array containing all the US States 2-Letter codes
states = """AK AL AR AZ CA CO CT DE FL GA HI IA ID IL IN KS KY LA MA MD ME
 MI MN MO MS MT NC ND NE NH NJ NM NV NY OH OK OR PA RI SC SD TN TX UT VA VT
 WA WI WV WY"""
# Array to hold the dictionary correspondent to each station
stations = []
# Looping through states
for state in states.split():
    
    clear_output(wait=True)
    print(f'Loading stations for state {state}')
    
    # Manipulating the uri to the proper network corresponding to the state
    uri = (
        "https://mesonet.agron.iastate.edu/"
        f"geojson/network/{state}_ASOS.geojson"
    )
    # Loading the data from the uri of the state
    data = urlopen(uri)
    # Appending the list of stations gathered, interpreted by json
    stations += json.load(data)["features"]
    
# This bit will clean up the dictionaries that are returned, 
# not important, but you can comment this seciton out and see how the
# original dictionaries look. Not all that pretty.
clear_output(wait=True)
print("Cleaning Up Station Information")
for station in stations:
    clear_output(wait=True)
    print(f"Cleaning Up Station Information for station {station['id']}")
    for key in station['properties'].keys():
        station[key] = station['properties'][key]
    del station['properties']
    station['lon'] = station['geometry']['coordinates'][0]
    station['lat'] = station['geometry']['coordinates'][1]
    del station['geometry']
    del station['type']
    del station['sid']
clear_output(wait=True)
print(f'{len(stations)} stations found!')

Cleaning Up Station Information for station UES


In [6]:
import numpy as np
import pandas as pd
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
import sys
sys.path.append(root_path)
# Utilities package
#from src.backend import utils

# A custom function that turns strings into ints, floats, dates, or, if all else fails, the original string.
def my_eval(s):
    """
    Converts a string into an integer, float, or date object, depending on the string's content. 
    If the conversion is unsuccessful, returns the original string.

    This function attempts to interpret a string in the following order:
    1. Tries to convert the string to an integer.
    2. If unsuccessful, tries to convert it to a float.
    3. If still unsuccessful, tries to convert it to a date.
    4. If all conversions fail, it returns the original string.

    Parameters:
    - s (str): The string to convert.

    Returns:
    - int, float, datetime.date, or str: The converted value or the original string if all conversions fail.
    """
    try:
        # Try to convert the string to a number (int or float)
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError:
            try:
                # Try to convert the string to a datetime date
                return datetime.datetime.strptime(s, '%Y-%m-%d').date()
            except ValueError:
                # If all conversions fail, return the original string
                return s
            
# Function that will interpret the utf-8 text data recovered and turn it into a dataframe
def text_to_dataframe(data):
    """
    Converts a text block of data into a Pandas DataFrame.

    The function processes a long string of data, where each line represents either a comment, 
    a header (with column names), or a row of data. Lines beginning with '#' are comments and 
    are ignored, as are empty lines. The first line of non-commented text is treated as the header, 
    providing the column names for the DataFrame. Subsequent lines of data are split by commas 
    and added to the respective columns.

    Each entry in the data is processed by the 'my_eval' function to attempt conversion into 
    int, float, or date types, falling back to the original string if necessary.

    Parameters:
    - data (str): The block of text data to convert.

    Returns:
    - DataFrame: A Pandas DataFrame representing the structured data.
    """

    # Dictionary that will become a dataframe
    data_values = {}
    # Looping thorugh lines of data
    for line in data.split('\n'):
        # Ignoring empty lines or commented-out lines
        if not line.startswith('#') and len(line) > 0:
            # The first non-commented line contains the data columns.
            # If there's nothing in the dictionary, it means that we're there, redefine the dictionary
            if len(data_values) == 0:
                # Start a dictionary where the keys are the columns, and entries are empty arrays (which we'll populate)
                data_values = {col:[] for col in line.split(',')}
            # If we're not in the columns line, we're in a line that contains data
            else:
                # Split data and append it to each entry of dictionary
                line_data = line.split(',')
                for i, col in enumerate(data_values.keys()):
                    data_values[col] += [my_eval(line_data[i])]
    return pd.DataFrame(data_values)

# Function to filter stations to ones that have any data in between startts and endts
def filter_stations(stations, startts, endts):
    """
    Filter the list of station dictionaries. Include a station if its operation 
    dates overlap with the provided startts and endts, even partially.

    Parameters:
    - stations (list of dicts): The list of station dictionaries.
    - startts (datetime): The start datetime.
    - endts (datetime): The end datetime.

    Returns:
    - list of dicts: The filtered list of stations.
    """
    filtered_stations = []

    for station in stations:
        # Parse the station's operation start and end dates.
        # It's important to handle cases where these dates are None or empty strings.
        archive_begin_str = station.get('archive_begin', '')
        archive_end_str = station.get('archive_end', '')

        try:
            archive_begin = datetime.datetime.strptime(archive_begin_str, '%Y-%m-%d') if archive_begin_str else None
        except ValueError:
            archive_begin = None

        try:
            # If archive_end is None or an empty string, we assume the station is still operational
            archive_end = datetime.datetime.strptime(archive_end_str, '%Y-%m-%d') if archive_end_str else datetime.datetime.max
        except ValueError:
            archive_end = None

        # Check if the station's operation dates overlap with the query dates.
        # The station is considered valid if its operation dates overlap even partially with the query dates.
        if archive_begin is not None and archive_end is not None:
            if (startts <= archive_end and endts >= archive_begin):
                filtered_stations.append(station)

    return filtered_stations

# Start and end dates for analysis
startts = datetime.datetime(2023, 1, 1)
endts = datetime.datetime(2023, 1, 2)
# Filtering stations to ones that have data in between startts and endtts
stations = filter_stations(stations, startts, endts)
# Base URL to manipulate
service = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&"
# Adding dates to URL in proper format
service += startts.strftime("year1=%Y&month1=%m&day1=%d&")
service += endts.strftime("year2=%Y&month2=%m&day2=%d&")
# Specifying the station to get data from
station = stations[0]
# These are the data columns that we care about, we'll ignore the rest
columns_of_interest = ['station', 'valid', 'lon', 'lat', 'tmpf',
                    'relh', 'drct', 'sknt', 'p01i', 'skyc1', 'skyc2',
                    'skyc3', 'skyc4', 'skyl1', 'skyl2', 'skyl3', 'skyl4',
                    'wxcodes', 'ice_accretion_1hr', 'metar']
# Building proper uri for the request
uri = f"{service}&station={station['id']}"

clear_output(wait=True)
print(f"Requesting data for station {station['id']} on base-url: {service}")
# Loading raw data for a single station
data = urlopen(uri, timeout=300).read().decode("utf-8")
# Interpreting data to a dataframe
data = text_to_dataframe(data)[columns_of_interest]

# This Section will slightly tweak some aspects of the data to make them nicer and more human-readable

# Interprets the 'valid' entry as a datetime object
data['valid'] = pd.to_datetime(data['valid'])
# Calculates the equivalent timestamp to each date
#data['timestamp'] = data['valid'].apply(utils.to_unix_timestamp)
# Whenever there's missing data, it's filled with an 'M', this code will replace that with NaNs
data = data.replace('M', np.nan)
# This code will create the column `wspd`, which is the measured wind speed in meters per second, instead of knots.
data['wspd'] = data['sknt']*0.514444
# Creates columns of the North and East decomposition of the wind vector
data['wind_E'] = np.sin(data['drct']/180*np.pi)*data['wspd']
data['wind_N'] = np.cos(data['drct']/180*np.pi)*data['wspd']
# Creating a dataframe for the stations found so we can assign an elevation to the data according to the station id.
stations_df = pd.DataFrame(stations)
# Assigning 'elevation' to each entry based on the station id
data = pd.merge(data, stations_df[['id', 'elevation']], left_on='station', right_on='id', how='left')
# Deleting 'id' column that is created as a side-product of the process above
data.drop('id', axis='columns', inplace=True)

# sorting data by date
data.sort_values(by='valid', ignore_index=True, inplace=True)
data

,station,valid,lon,lat,tmpf,relh,drct,sknt,p01i,skyc1,...,skyl2,skyl3,skyl4,wxcodes,ice_accretion_1hr,metar,wspd,wind_E,wind_N,elevation
0,PADK,2023-01-01 00:00:00,-176.646,51.878,NaN,NaN,360.0,11.0,NaN,SCT,...,2300.0,3500.0,NaN,NaN,NaN,PADK 010000Z AUTO 36011KT SCT016 BKN023 BKN035...,5.658884,-1.386027e-15,5.658884,4.0
1,PADK,2023-01-01 00:05:00,-176.646,51.878,NaN,NaN,360.0,11.0,NaN,SCT,...,2300.0,2900.0,NaN,NaN,NaN,PADK 010005Z AUTO 36011KT SCT016 BKN023 BKN029...,5.658884,-1.386027e-15,5.658884,4.0
2,PADK,2023-01-01 00:10:00,-176.646,51.878,NaN,NaN,350.0,8.0,NaN,SCT,...,2300.0,3500.0,NaN,NaN,NaN,PADK 010010Z AUTO 35008KT SCT016 BKN023 OVC035...,4.115552,-7.146581e-01,4.053028,4.0
3,PADK,2023-01-01 00:15:00,-176.646,51.878,NaN,NaN,330.0,5.0,0.01,FEW,...,2100.0,3500.0,NaN,NaN,NaN,PADK 010015Z AUTO 33005KT FEW014 BKN021 OVC035...,2.572220,-1.286110e+00,2.227608,4.0
4,PADK,2023-01-01 00:20:00,-176.646,51.878,NaN,NaN,330.0,5.0,0.01,FEW,...,2100.0,3500.0,NaN,NaN,NaN,PADK 010020Z AUTO 33005KT FEW012 BKN021 OVC035...,2.572220,-1.286110e+00,2.227608,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,PADK,2023-01-01 23:40:00,-176.646,51.878,NaN,NaN,310.0,13.0,0.01,SCT,...,2600.0,3300.0,NaN,NaN,NaN,PADK 012340Z AUTO 31013KT SCT016 BKN026 OVC033...,6.687772,-5.123131e+00,4.298817,4.0
330,PADK,2023-01-01 23:45:00,-176.646,51.878,NaN,NaN,330.0,16.0,0.01,SCT,...,2800.0,3400.0,NaN,NaN,NaN,PADK 012345Z AUTO 33016KT SCT014 BKN028 OVC034...,8.231104,-4.115552e+00,7.128345,4.0
331,PADK,2023-01-01 23:50:00,-176.646,51.878,NaN,NaN,320.0,14.0,0.01,SCT,...,2400.0,3500.0,NaN,NaN,NaN,PADK 012350Z AUTO 32014KT SCT015 BKN024 OVC035...,7.202216,-4.629495e+00,5.517218,4.0
332,PADK,2023-01-01 23:55:00,-176.646,51.878,NaN,NaN,330.0,14.0,0.01,FEW,...,2400.0,3500.0,NaN,NaN,NaN,PADK 012355Z AUTO 33014KT FEW015 BKN024 OVC035...,7.202216,-3.601108e+00,6.237302,4.0


In [22]:
data = pd.read_csv('../data/weather/1672617600_1672704000.csv')

/var/folders/5c/4dc4m3p13_gd8160xmlxf97c0000gn/T/ipykernel_14397/798714318.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/weather/1672617600_1672704000.csv')


In [34]:
metardf=data[['metar']].copy()

In [35]:
metardf

,metar
0,K04V 010013Z AUTO 31009KT 10SM CLR 00/M03 A298...
1,K04V 010033Z AUTO 31008KT 10SM CLR 00/M03 A298...
2,K04V 010053Z AUTO 29007KT 10SM CLR 00/M03 A299...
3,K04V 010113Z AUTO 30011G14KT 10SM CLR M01/M03 ...
4,K04V 010133Z AUTO 31006KT 10SM SCT100 M01/M03 ...
...,...
415940,KZZV 012340Z AUTO 33002KT 10SM BKN030 09/08 A3...
415941,KZZV 012345Z AUTO 00000KT 10SM SCT022 BKN030 0...
415942,KZZV 012350Z AUTO 00000KT 10SM BKN022 OVC030 0...
415943,KZZV 012353Z AUTO 00000KT 10SM BKN022 OVC030 0...


In [36]:
metardf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415945 entries, 0 to 415944
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   metar   415945 non-null  object
dtypes: object(1)
memory usage: 3.2+ MB


In [41]:
metarlist = metardf['metar'].tolist()
metarlist = ['METAR ' + m for m in metarlist]
metarlist

['METAR K04V 010013Z AUTO 31009KT 10SM CLR 00/M03 A2989 RMK AO2',
 'METAR K04V 010033Z AUTO 31008KT 10SM CLR 00/M03 A2989 RMK AO2',
 'METAR K04V 010053Z AUTO 29007KT 10SM CLR 00/M03 A2990 RMK AO2',
 'METAR K04V 010113Z AUTO 30011G14KT 10SM CLR M01/M03 A2990 RMK AO2',
 'METAR K04V 010133Z AUTO 31006KT 10SM SCT100 M01/M03 A2991 RMK AO2',
 'METAR K04V 010153Z AUTO 28005KT 10SM SCT100 M02/M04 A2992 RMK AO2',
 'METAR K04V 010213Z AUTO 31009KT 10SM CLR M03/M05 A2992 RMK AO2',
 'METAR K04V 010233Z AUTO 27004KT 10SM CLR M03/M05 A2993 RMK AO2',
 'METAR K04V 010253Z AUTO 00000KT 10SM CLR M04/M06 A2993 RMK AO2',
 'METAR K04V 010313Z AUTO 29003KT 10SM CLR M05/M06 A2994 RMK AO2',
 'METAR K04V 010333Z AUTO 31005KT 10SM SCT090 M04/M05 A2994 RMK AO2',
 'METAR K04V 010353Z AUTO 29005KT 10SM BKN090 M04/M05 A2994 RMK AO2',
 'METAR K04V 010413Z AUTO 00000KT 10SM BKN080 M04/M06 A2995 RMK AO2',
 'METAR K04V 010433Z AUTO 00000KT 10SM SCT080 M05/M06 A2995 RMK AO2',
 'METAR K04V 010453Z AUTO 00000KT 10SM CLR M

In [43]:
from IPython.display import clear_output

In [58]:
# ExaIPythone METAR data
#metar_data = 'METAR KATL 112351Z 09004KT 10SM FEW250 13/M02 A3013 RMK AO2 SLP204 T01281017 10139 20128 51006'

df=pd.DataFrame()
# Parse METAR data to a pandas DataFrame
for i in range(len(data[:10000])):
    if i % 100 == 0:
        clear_output(wait=True)
        print(f'{i}/{len(data)}')
    newrow = parse_metar_to_dataframe(metarlist[i])
    df = pd.concat([df, newrow], ignore_index=True)

# Display the DataFrame
print(df)

9900/415945
     station_id  latitude  longitude  elevation           date_time  \
0          K04V     38.10    -106.17     2385.0 2023-11-01 00:13:00   
1          K04V     38.10    -106.17     2385.0 2023-11-01 00:33:00   
2          K04V     38.10    -106.17     2385.0 2023-11-01 00:53:00   
3          K04V     38.10    -106.17     2385.0 2023-11-01 01:13:00   
4          K04V     38.10    -106.17     2385.0 2023-11-01 01:33:00   
...         ...       ...        ...        ...                 ...   
9995       K8B0     44.97     -70.70      554.0 2023-11-01 16:15:00   
9996       K8B0     44.97     -70.70      554.0 2023-11-01 16:35:00   
9997       K8B0     44.97     -70.70      554.0 2023-11-01 16:55:00   
9998       K8B0     44.97     -70.70      554.0 2023-11-01 17:15:00   
9999       K8B0     44.97     -70.70      554.0 2023-11-01 17:35:00   

      wind_direction  wind_speed  wind_gust  visibility current_wx1  ...  \
0              310.0         9.0        NaN   16093.440    

In [32]:
# data['station_id'] = data['station']

In [78]:
newdf=pd.concat([metardf, df], axis=1)
np.unique(newdf['highest_cloud_level'].astype(str))
newdf[['']].dropna()

,low_cloud_level,medium_cloud_level,high_cloud_level,highest_cloud_level,station_id
1439,1800.0,2800.0,3500.0,7000.0,K15M
1440,1500.0,2800.0,3500.0,6500.0,K15M
3872,500.0,1800.0,6000.0,12000.0,K2DP
3873,1800.0,6000.0,12000.0,16000.0,K2DP
3879,800.0,1700.0,2300.0,3500.0,K2DP


In [94]:
newdf[['current_wx1','current_wx1_symbol']].dropna().drop_duplicates()

,current_wx1,current_wx1_symbol
124,BR,10.0
187,FZFG,49.0
318,-SN,71.0
444,FG,45.0
1170,-RA,61.0
1304,-DZ,51.0
1340,HZ,5.0
1871,-PL,79.0
2500,RA,63.0
2524,UP,141.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415945 entries, 0 to 415944
Data columns (total 32 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   metar                      415945 non-null  object        
 1   station_id                 10000 non-null   object        
 2   latitude                   9219 non-null    float64       
 3   longitude                  9219 non-null    float64       
 4   elevation                  9219 non-null    float64       
 5   date_time                  10000 non-null   datetime64[ns]
 6   wind_direction             9641 non-null    float64       
 7   wind_speed                 9786 non-null    float64       
 8   wind_gust                  536 non-null     float64       
 9   visibility                 9805 non-null    float64       
 10  current_wx1                3368 non-null    object        
 11  current_wx2                27 non-null      object  

In [ ]:
OKTAS